In [12]:
from functions_pt import *
import pickle
from torch.optim import Adam

In [10]:
embed_size = 128
dropout = 0.1
multiple_factor = 1
layers = 2
batch_size = 64
agg = 'mean' # mean,sum,min,max
lr = 1e-4
clip = 1
epochs = 10

In [3]:
# load data
ys = np.load('ys.npy')
weights = np.load('weights.npy')
length = np.load('length.npy')
words_embed = np.load('possible_words_embed.npy')
with open('indexes.pkl', 'rb') as f:
    indexes = pickle.load( f)

# train/val split
np.random.seed(0)
train_idx = np.random.rand(ys.shape[0])>0.2
val_idx = np.logical_not(train_idx)

ys_train, ys_val = ys[train_idx], ys[val_idx]
weights_train, weights_val = weights[train_idx], weights[val_idx]
length_train, length_val = length[train_idx], length[val_idx]
indexes_train = [idx for idx,bol in zip(indexes,train_idx) if bol]
indexes_val = [idx for idx,bol in zip(indexes,train_idx) if not bol]

# set up dataloader
dataset_train = CustomDataset(ys_train,length_train,indexes_train,words_embed)
sampler = WeightedRandomSampler(weights_train, 50000)
dataset_train = DataLoader(dataset_train, batch_size=batch_size, sampler=sampler,collate_fn=collate)

dataset_val = CustomDataset(ys_val,length_val,indexes_val,words_embed)
sampler = WeightedRandomSampler(weights_val, 10000)
dataset_val = DataLoader(dataset_val, batch_size=batch_size, sampler=sampler,collate_fn=collate)

In [4]:
model = pointNet(layers,embed_size,agg,dropout,multiple_factor).to('cuda')
opt = Adam([    {'params': [p for p in model.parameters() if p is not model.w]},\
                {'params': model.w, 'lr': lr/4}
            ],lr=lr)

In [22]:
model = train(opt,model,epochs,dataset_train,dataset_val,model.parameters(),clip)

epoch:0, train_loss: +1.361, val_loss: +0.659 

epoch:1, train_loss: +1.005, val_loss: +0.587 

epoch:2, train_loss: +0.737, val_loss: +0.448 

epoch:3, train_loss: +0.535, val_loss: +0.323 

epoch:4, train_loss: +0.385, val_loss: +0.208 

epoch:5, train_loss: +0.279, val_loss: +0.172 

epoch:6, train_loss: +0.204, val_loss: +0.108 

epoch:7, train_loss: +0.148, val_loss: +0.083 

epoch:8, train_loss: +0.104, val_loss: +0.077 

epoch:9, train_loss: +0.069, val_loss: +0.035 

Training completed in 159.48054599761963s


In [33]:
w = dataset_val.dataset.words_embed
l = torch.tensor([2309.0])

In [43]:
model((torch.tensor(w).to('cuda'),torch.tensor(l).to('cuda')))

/tmp/ipykernel_83420/3274103529.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model((torch.tensor(w).to('cuda'),torch.tensor(l).to('cuda')))


tensor([2.7551], device='cuda:0', grad_fn=<AddBackward0>)